In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import scipy.spatial.distance
# from sentence_transformers import SentenceTransformer, models

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW,RobertaModel
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer,AutoModel
import pickle
model_name = 'microsoft/deberta-base'
pooling_mode = 'mean'

In [ ]:
test_label = '/content/drive/MyDrive/input/testdata/STS.gs.images.txt'
test_data= '/content/drive/MyDrive/input/testdata/STS.input.images.txt'
train_label = '/content/drive/MyDrive/input/traindata/STS.gs.OnWN.txt'
train_data = '/content/drive/MyDrive/input/traindata/STS.input.OnWN.txt'
train_label2 = '/content/drive/MyDrive/input/testdata/STS.gs.deft-news.txt'
train_data2 = '/content/drive/MyDrive/input/testdata/STS.input.deft-news.txt'
train_label3 = '/content/drive/MyDrive/input/testdata/STS.gs.deft-forum.txt'
train_data3 = '/content/drive/MyDrive/input/testdata/STS.input.deft-forum.txt'
train_label4 = '/content/drive/MyDrive/input/testdata/STS.gs.headlines.txt'
train_data4 = '/content/drive/MyDrive/input/testdata/STS.input.headlines.txt'
train_label5 = '/content/drive/MyDrive/input/testdata/STS.gs.SMTeuroparl.txt'
train_data5 = '/content/drive/MyDrive/input/testdata/STS.input.SMTeuroparl.txt'
train_label6 = '/content/drive/MyDrive/input/testdata/STS.gs.MSRvid.txt'
train_data6 = '/content/drive/MyDrive/input/testdata/STS.input.MSRvid.txt'
train_label7 = '/content/drive/MyDrive/input/testdata/STS.gs.MSRpar.txt'
train_data7 = '/content/drive/MyDrive/input/testdata/STS.input.MSRpar.txt'
#output_path = './output/STS.output.images.txt'
from nltk.stem import *

#ファイル入力
def file_input(input_path,mode=None):
    with open(input_path,encoding='UTF-8-sig') as f:
        s = f.read()
    s = s.strip().replace('\t','\n')
    if mode == 'data':
      s = s.replace('.','')
    sentences = s.split('\n')
    return sentences

def stem(sentences):
  ps = PorterStemmer()
  for i in range(len(sentences)):
    token = sentences[i].split(' ')
    singles = [ps.stem(plural) for plural in token]
    sentences[i] = token
  return sentences


def sub_char(sentences:list):
    new_sentenes = []
    for sentence in sentences:
        new_sentence = []
        for word in sentence:
            w = word.replace(':','').replace(';','').replace('(','').replace(')','').replace(',','').replace('.','')
            new_sentence.append(w)
        new_sentenes.append(new_sentence)
    return new_sentenes

def list_to_seq(sentences):
  new_sentences = []
  for sentnece in sentences:
    new_sentence = ' '.join(sentnece)
    new_sentences.append(new_sentence)
  return new_sentences

def str_to_float_on_list(labels):
  new_labels = []
  for label in labels:
    new_labels.append(float(label))
  return new_labels

test_sentences = file_input(test_data,'data')
stem(test_sentences)
test_sentences = list_to_seq(test_sentences)
print(test_sentences)

test_labels_raw = file_input(test_label)
test_labels_raw = str_to_float_on_list(test_labels_raw)
print(test_labels_raw)

train_sentences = file_input(train_data,'data')
stem(train_sentences)
train_sentences = list_to_seq(train_sentences)
print(train_sentences)

train_labels = file_input(train_label)
train_labels = str_to_float_on_list(train_labels)
print(train_labels)

train_sentences2 = file_input(train_data2,'data')
stem(train_sentences2)
train_sentences2 = list_to_seq(train_sentences2)
print(train_sentences2)

train_labels2 = file_input(train_label2)
train_labels2 = str_to_float_on_list(train_labels2)
print(train_labels2)

train_sentences3 = file_input(train_data3,'data')
stem(train_sentences3)
train_sentences3 = list_to_seq(train_sentences3)
print(train_sentences3)

train_labels3 = file_input(train_label3)
train_labels3 = str_to_float_on_list(train_labels3)
print(train_labels3)

train_sentences4 = file_input(train_data4,'data')
stem(train_sentences4)
train_sentences4 = list_to_seq(train_sentences4)
print(train_sentences4)

train_labels4 = file_input(train_label4)
train_labels4 = str_to_float_on_list(train_labels4)
print(train_labels4)

train_sentences5 = file_input(train_data5,'data')
stem(train_sentences5)
train_sentences5 = list_to_seq(train_sentences5)
print(train_sentences5)

train_labels5 = file_input(train_label5)
train_labels5 = str_to_float_on_list(train_labels5)
print(train_labels5)

train_sentences6 = file_input(train_data6,'data')
stem(train_sentences6)
train_sentences6 = list_to_seq(train_sentences6)
print(train_sentences6)

train_labels6 = file_input(train_label6)
train_labels6 = str_to_float_on_list(train_labels6)
print(train_labels6)

train_sentences7 = file_input(train_data7,'data')
stem(train_sentences7)
train_sentences7 = list_to_seq(train_sentences7)
print(train_sentences7)

train_labels7 = file_input(train_label7)
train_labels7 = str_to_float_on_list(train_labels7)
print(train_labels7)

# print(len(test_sentences),len(test_labels),len(train_sentences),len(train_labels))
# print(len(train_sentences2),len(train_labels2),len(train_sentences3),len(train_labels3))

['A cat standing on tree branches', 'A black and white cat is high up on tree branches', 'Two green and white trains sitting on the tracks', 'Two green and white trains on tracks', 'A small white cat with glowing eyes standing underneath a chair', 'A white cat stands on the floor', 'A large boat in the water at the marina', 'A large boat on the sea', 'a bus driving in a street', 'Red double decker bus driving down street', 'The lamb is looking at the camera', 'A small bird standing on a log at the waters edge', 'A passenger train waiting in a station', 'A passenger train sits in the station', 'a woman at a dinner table writing on her notebook', 'Woman at table busy with something', 'An Apple computer sitting on the floor', 'A Macintosh computer sitting on the floor', "A close-up of a brown horse's head", "A side view of a horse's head", 'A group of people eat at a table outside', 'Group of elderly people sitting around a table', 'The curious boys look in the shed', 'Two small boys in a

In [ ]:
train_sentences = train_sentences +  train_sentences2 +  train_sentences3 + train_sentences4 +  train_sentences5 +  train_sentences6 +  train_sentences7
train_labels_raw = train_labels + train_labels2 + train_labels3 + train_labels4 + train_labels5 + train_labels6 +  train_labels7

In [ ]:
train_sentences1 = []
train_sentences2 = []
test_sentences1 = []
test_sentences2 = []
train_labels =[]
test_labels = []
for i in range(int(len(train_sentences)/2)):
  train_sentences1.append(train_sentences[2*i])
  train_sentences2.append(train_sentences[2*i+1])
for j in range(int(len(test_sentences)/2)):
  test_sentences1.append(test_sentences[2*j])
  test_sentences2.append(test_sentences[2*j+1])
for k in range(int(len(train_labels_raw))):
  train_labels.append(train_labels_raw[k]/5.0)
for l in range(int(len(test_labels_raw))):
  test_labels.append(test_labels_raw[l]/5.0)

In [ ]:
print(len(train_sentences1))
print(train_sentences1[0],train_sentences2[0])
print(len(test_sentences1))
print(test_sentences1[0],test_sentences2[0])

4484
the activity of learning or being trained the gradual process of acquiring knowledge
750
A cat standing on tree branches A black and white cat is high up on tree branches


In [ ]:
print(test_labels)
print(test_labels_raw)

[0.72, 0.8800000000000001, 0.52, 0.64, 0.8, 0.0, 0.96, 0.6799999999999999, 0.76, 0.6799999999999999, 0.5599999999999999, 0.5599999999999999, 0.76, 0.27999999999999997, 0.8, 0.76, 0.44000000000000006, 0.0, 0.8800000000000001, 0.8, 0.76, 0.6, 0.8800000000000001, 0.12, 0.64, 0.52, 0.72, 0.52, 0.12, 0.0, 0.52, 0.27999999999999997, 0.8400000000000001, 0.0, 0.08, 0.8, 0.45, 0.24, 0.8, 0.64, 0.76, 0.8, 1.0, 0.2, 0.0, 0.8800000000000001, 0.08, 0.8, 0.24, 0.32, 0.0, 0.6, 0.64, 0.27999999999999997, 0.64, 0.76, 0.08, 0.36, 0.76, 0.64, 0.64, 0.12, 0.44000000000000006, 0.6799999999999999, 0.36, 0.8800000000000001, 0.48, 0.72, 0.16, 0.12, 0.0, 0.16, 0.8400000000000001, 0.72, 0.48, 0.72, 0.64, 0.04, 0.48, 0.16, 0.8, 0.9199999999999999, 0.6799999999999999, 0.08, 0.6799999999999999, 0.12, 0.76, 0.76, 0.4, 0.48, 0.6, 0.76, 0.4, 0.52, 0.44000000000000006, 1.0, 0.4, 0.8400000000000001, 0.76, 0.16, 0.0, 0.44000000000000006, 0.72, 0.72, 0.0, 0.9199999999999999, 0.0, 0.8800000000000001, 0.6799999999999999, 0

In [ ]:
print(train_sentences1)
print(train_sentences2)
print(train_labels)

['the activity of learning or being trained', 'thin metal strips used in typesetting', 'falcon: abandon intended game and pursue lesser prey', 'the act of designating a role to someone', 'make pure or free from sin', 'leave undone or leave out', 'a social set or clique of friends', 'The upper part of a column', 'sink to a lower level; descend', 'create code or computer programs', 'release from confinement, obligation, or a burden', '(Cause to) sully the good name and reputation of', 'a company that provides public service commodities', 'Physically extend or spread outward from a center or focus or inward towards a center; physically spread from a start point', 'Distribute systematically or strategically; initiate', 'create something in the mind', '(cause to) become less clear or indistinct', 'exempt or free from a burden, duty, or requirement', 'a straight length of road or racetrack', '(cause to) emit a brief burst of light', 'stretch to the utmost, causing stress or injury', 'move in

In [ ]:
class STSDataset(Dataset):
    def __init__(self, sent1, sent2, labels, tokenizer, max_len):
        self.sent1 = sent1
        self.sent2 = sent2
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        sent1 = str(self.sent1[idx])
        sent2 = str(self.sent2[idx])
        label = float(self.labels[idx])

        encoding1 = self.tokenizer(
            sent1,
            return_tensors='pt',
            max_length=self.max_len,
            padding='max_length',
            truncation=True
        )
        encoding2 = self.tokenizer(
            sent2,
            return_tensors='pt',
            max_length=self.max_len,
            padding='max_length',
            truncation=True
        )

        return {
            'input_ids1': encoding1['input_ids'].flatten(),
            'attention_mask1': encoding1['attention_mask'].flatten(),
            'input_ids2': encoding2['input_ids'].flatten(),
            'attention_mask2': encoding2['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.float)
        }

In [ ]:


MAX_LEN = 128  # 入力トークンの最大長
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
EPOCHS = 5
criterion = torch.nn.MSELoss()
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
train_dataset = STSDataset(train_sentences1, train_sentences2, train_labels, tokenizer, MAX_LEN)
test_dataset = STSDataset(test_sentences1, test_sentences2, test_labels, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
import scipy.spatial.distance
from drive.MyDrive.my_pearson import output_pearson
def pooling(last_hidden_state,mode):
  if (mode=='mean'):
    sentence_vector = last_hidden_state.mean(dim=1)
  else :
    sentence_vector = last_hidden_state[:,0,:]
  return sentence_vector

def get_vecs(model,sent1,sent2):
  model.eval()
  encoded_inputs1 = tokenizer(sent1, padding=True, truncation=True, return_tensors='pt',max_length=128).to(device)
  encoded_inputs2 = tokenizer(sent2, padding=True, truncation=True, return_tensors='pt',max_length=128).to(device)
  with torch.no_grad():
      outputs1 = model(**encoded_inputs1)
      outputs2 = model(**encoded_inputs2)

  last_hidden_state1 = outputs1.last_hidden_state
  last_hidden_state2 = outputs2.last_hidden_state
  # print(last_hidden_state1.size())
  # print(last_hidden_state2.size())
  #sentence_vectors1 = last_hidden_state1[:, 0, :]  # 例として、各文の[CLS]トークンのベクトルを取得
  # sentence_vectors1 = last_hidden_state1.mean(dim=1)
  #sentence_vectors2 = last_hidden_state2[:, 0, :]
  # sentence_vectors2 = last_hidden_state2.mean(dim=1)
  sentence_vectors1 = pooling(last_hidden_state1,pooling_mode)
  sentence_vectors2 = pooling(last_hidden_state2,pooling_mode)
  return sentence_vectors1,sentence_vectors2

def calc_sim(vecs1,vecs2):
  cos_list = []
  #768
  for i in range(len(vecs1)):
    nump1 = vecs1[i].detach().cpu().numpy()
    nump2 = vecs2[i].detach().cpu().numpy()
    cos_list.append( 1 - scipy.spatial.distance.cosine(nump1,nump2))

  return cos_list

def predict(model,sent1,sent2,labels):
  vecs1, vecs2 = get_vecs(model,sent1,sent2)
  # print(vecs1.size())
  # print(vecs2.size())
  cos_list = calc_sim(vecs1,vecs2)
  # print(cos_list)
  # print(len(cos_list))
  # print(len(labels))
  p = output_pearson(labels,cos_list)
  return p

In [ ]:
p =predict(model,test_sentences1,test_sentences2,test_labels)
pearsons = [p]
print(p)

0.26020174721119327


In [ ]:
sentence1 = ['I am a human','I am a ginius']
sentence2 = ['You are a human','Apple is red']
vecs1,vecs2 = get_vecs(model,sentence1,sentence2)
print(vecs1,vecs2)
print(vecs1.size(),vecs2.size())
print(calc_sim(vecs1,vecs2))

tensor([[-0.1101, -0.2779, -0.2074,  ...,  0.4891, -0.0245, -0.0501],
        [ 0.2964, -0.4590, -0.2777,  ...,  0.6490, -0.2480, -0.1877]],
       device='cuda:0') tensor([[-0.0467, -0.5324, -0.3032,  ...,  0.5828, -0.1605, -0.0719],
        [ 0.0550, -0.1762, -0.1368,  ...,  0.4672, -0.0411,  0.0374]],
       device='cuda:0')
torch.Size([2, 768]) torch.Size([2, 768])
[0.9656157493591309, 0.920568585395813]


In [ ]:
MODEL_PATH = '/content/drive/MyDrive/output/RoBERTa'
losses = [0.29965]
print(pearsons[0])
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids1 = batch['input_ids1'].to(device)
        attention_mask1 = batch['attention_mask1'].to(device)
        input_ids2 = batch['input_ids2'].to(device)
        attention_mask2 = batch['attention_mask2'].to(device)
        labels = batch['label'].to(device)
        #print(input_ids1.size)
        optimizer.zero_grad()
        output1 = model(input_ids=input_ids1, attention_mask=attention_mask1)
        output2 = model(input_ids=input_ids2, attention_mask=attention_mask2)
        #sentence_embeddings1 = output1.last_hidden_state.mean(dim=1)  # 文の特徴量を平均
        sentence_embeddings1 = pooling(output1.last_hidden_state,mode=pooling_mode)
        sentence_embeddings2 = pooling(output2.last_hidden_state,mode=pooling_mode)
        #sentence_embeddings1 = output1.last_hidden_state[:, 0, :]
        #sentence_embeddings2 = output2.last_hidden_state.mean(dim=1)
        #sentence_embeddings2 = output2.last_hidden_state[:, 0, :]
        #print(sentence_embeddings1.size())
        similarity = torch.cosine_similarity(sentence_embeddings1, sentence_embeddings2, dim=-1).unsqueeze(0)
        loss = criterion(similarity, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    average_loss = total_loss / len(train_loader)
    losses.append(average_loss)
    with open(MODEL_PATH+f'/loss{epoch}.pickle',mode='wb') as f:
       pickle.dump(losses,f)
    model.eval()
    p = predict(model,test_sentences1,test_sentences2,test_labels)
    print(p)
    pearsons.append(p)
    with open(MODEL_PATH+f'/pearson{epoch}.pickle',mode='wb') as f:
       pickle.dump(pearsons,f)
    print(f'Epoch {epoch + 1}/{EPOCHS}, Loss: {average_loss:.4f}')

0.26020174721119327


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([1, 16])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([1, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


0.17396512427281327
Epoch 1/5, Loss: 0.2272
0.10586549121936466
Epoch 2/5, Loss: 0.2281
0.06267752053485581
Epoch 3/5, Loss: 0.2283


In [ ]:
# with open('/content/drive/MyDrive/output/RoBERTa/loss4.pickle',mode='br') as f:
#   losses = pickle.load(f)
# with open('/content/drive/MyDrive/output/RoBERTa/pearson4.pickle',mode='br') as f:
#   pearsons = pickle.load(f)

In [ ]:
import matplotlib.pyplot as plt
losses[0] = 0.9432
peason = pearsons
loss = losses
print(peason)
print(loss)
x = [i for i in range(len(peason))]
print(x)

fig,ax1 =  plt.subplots()

color = 'tab:blue'
ax1.set_ylabel('total_loss',color=color)
ax1.plot(x,loss,color=color)
ax1.tick_params(axis='y',labelcolor=color)

ax2 = ax1.twinx()

color='tab:red'
ax2.set_ylabel('pearson',color=color)
ax2.plot(x,peason,color=color)
ax2.tick_params(axis='y',labelcolor=color)

color = 'tab:gray'
ax1.set_xlabel('epoch',color=color)
ax1.tick_params(axis='x',labelcolor=color)

fig.tight_layout()
plt.savefig(f'/content/drive/MyDrive/output/graph_{model_name}_{pooling_mode}.png')
plt.show()

In [ ]:
# 文のリストを用意する
sentences = [
    "This is sentence 1.",
    "Another example sentence.",
    "I am a human"
    # 他の文を追加
]

# 文のリストをトークン化してエンコード
encoded_inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt').to(device)
# モデルに入力を渡して出力を得る
model.eval()
with torch.no_grad():
    outputs = model(**encoded_inputs)

# 最終層の隠れ状態を取得
last_hidden_state = outputs.last_hidden_state

# 文のリストの各文に対応するベクトルを取得
sentence_vectors = last_hidden_state[:, 0, :]  # 例として、各文の[CLS]トークンのベクトルを取得

# sentence_vectorsは各文のベクトルとして利用できます
print(sentence_vectors)